In [1]:
import pandas as pd
import numpy as np
import glob
import re
import win32com.client as win32
import os
win32c = win32.constants

In [2]:
# Read all csv files
files = glob.glob("data/*.csv")

df = pd.concat((pd.read_csv(file) for file in files))

# Clean up headers
df = df[pd.isna(df['Instrument']) == False]
df.reset_index(drop=True, inplace=True)

In [3]:
# Calculate some columns
df['Credit/Debit'] = df['Quantity'] * df['Price'] * -1
date_regex = r"(\d{2}) (\w{3}) (\d{4})"
matches = df['Trigger Date'].apply(lambda row: re.search(date_regex, row))
df['Date'] = matches.apply(lambda match: match.group(1))
df['Month'] = matches.apply(lambda match: match.group(2))
df['Year'] = matches.apply(lambda match: match.group(3))

# Save output
output_file = 'output.xlsx'
os.remove(output_file)
writer = pd.ExcelWriter(output_file) 
df.to_excel(writer, sheet_name = 'Raw', index=False)

# Auto adjust column width
for column in df:
    column_length = max(df[column].astype(str).map(len).max(), len(column))
    col_idx = df.columns.get_loc(column)
    writer.sheets['Raw'].set_column(col_idx, col_idx, column_length)

writer.save()

In [4]:
# Analysis
df.pivot_table(index=['Year', 'Month'], values=['Credit/Debit'], aggfunc='sum')

Credit/Debit
Year Month              
2021 Apr    13711.247101
     Aug    13662.499847
     Dec    13601.250420
     Feb    -8731.248856
     Jan    14833.750916
     Jul    -9417.498550
     Jun   -14436.247101
     Mar    -3133.748550
     May    25036.249695
     Nov    34527.497711
     Oct    19089.998741
     Sep    23440.000343

January 2021
